In [5]:
import os # to List-dir 
import numpy as np 

from PIL import Image # python image lib 
from numpy import asarray 
import matplotlib . pyplot as pit 
import matplotlib. image as mpimg 
from sklearn import linear_model 
import glob
from sklearn.metrics import confusion_matrix, accuracy_score

In [17]:
def Preprocess_Images(folder_path):
    
    #create an iterable list of training images
    file_names = glob.glob(folder_path)
    
    #define our empty  x and y arrays
    x = np.empty((len(file_names),4096))
    y = np.empty(len(file_names),dtype=int)
    
    # Load images, scale, and store into a dataset 
    for i in range(len(file_names)): 
        #load path for image i
        path = file_names[i]
        #load image and convert to grayscale
        im = Image.open(path).convert('L') 
        #crop, resize and convert to grayscale
        width, height = im.size
        side = min(width,height)
        im = im.crop((width/2-side/2,height/2-side/2, width/2+side/2,height/2+side/2))
        im = im.resize((64,64))
        im_array = np.asarray(im)
        x[i,:] = im_array.ravel().reshape(1,-1)
        #assign categorical label to each image based on file name
        if 'cir' in path:
            y[i] = 0
        elif 'squ' in path:
            y[i] = 1
        elif 'rec' in path:
            y[i] = 2
    
    return x,y

def get_accuracy(actual, pred):
    accuracy = (actual == pred).sum() / float(len(actual))
    return accuracy

In [18]:
#inputting training data into model, update file path to training folder
X,y = Preprocess_Images(folder_path = r'Data\training\*.png')
print("x shape: " ,X.shape)
print("y shape: " ,y.shape)


x shape:  (81, 4096)
y shape:  (81,)


In [19]:
import matplotlib.pyplot as plt
import numpy as np
import nnfs

from nnfs.datasets import vertical_data, spiral_data
nnfs.init()

# Dense layer
class Layer_Dense:
    # Layer initialization
    def __init__(self, n_inputs, n_neurons):
        # Initialize weights and biases
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
        # Forward pass
    def forward(self, inputs):
        # Remember input values
        self.inputs = inputs
        # Calculate output values from inputs, weights and biases
        self.output = np.dot(inputs, self.weights) + self.biases
        # Backward pass
    def backward(self, dvalues):
        # Gradients on parameters
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        # Gradient on values
        self.dinputs = np.dot(dvalues, self.weights.T)


class Activation_ReLU:
    def forward(self, inputs):
        # Remember input values
        self.inputs = inputs
        # Calculate output values from inputs
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):

        # Since we need to modify the original variable,
        # let's make a copy of the values first
        self.dinputs = dvalues.copy()
        # Zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0

class Activation_Softmax:
    def forward(self, inputs):
        # Remember input values
        self.inputs = inputs
        # Get unnormalized probabilities
        exp_values = np.exp(inputs - np.max(inputs, axis=1,
                                            keepdims=True))
        # Normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1,
                                            keepdims=True)
        self.output = probabilities
    # Backward pass
    def backward(self, dvalues):
        # Create uninitialized array
        self.dinputs = np.empty_like(dvalues)
        # Enumerate outputs and gradients
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            # Flatten output array
            single_output = single_output.reshape(-1, 1)
            # Calculate Jacobian matrix of the output and
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            # Calculate sample-wise gradient
            # and add it to the array of sample gradients
            self.dinputs[index] = np.dot(jacobian_matrix,single_dvalues)
            
class Activation_Softmax_Loss_CategoricalCrossentropy():
    # Creates activation and loss function objects
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossentropy()
    # Forward pass
    def forward(self, inputs, y_true):
        # Output layer's activation function
        self.activation.forward(inputs)
        # Set the output
        self.output = self.activation.output
        # Calculate and return loss value
        return self.loss.calculate(self.output, y_true)
    # Backward pass
    def backward(self, dvalues, y_true):
        # Number of samples
        samples = len(dvalues)
        # If labels are one-hot encoded,
        # turn them into discrete values
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)
        # Copy so we can safely modify
        self.dinputs = dvalues.copy()
        # Calculate gradient
        self.dinputs[range(samples), y_true] -= 1
        # Normalize gradient
        self.dinputs = self.dinputs / samples
        
class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss
    
class Loss_CategoricalCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        # probabilities for target values - only if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        # mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods
    # Backward pass
    def backward(self, dvalues, y_true):
        # Number of samples
        samples = len(dvalues)
        # Number of labels in every sample
        # We'll use the first sample to count them
        labels = len(dvalues[0])
        # If labels are sparse, turn them into one-hot vector
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]
        # Calculate gradient
        self.dinputs = -y_true / dvalues
        # Normalize gradient
        self.dinputs = self.dinputs / samples

class Optimizer_SGD:
    def __init__(self, learning_rate=1.0, decay=0.0, momentum=0.0):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))
    def update_params(self, layer):
        if self.momentum:
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)
            weight_updates = self.momentum * layer.weight_momentums - self.current_learning_rate * layer.dweights
            layer.weight_momentums = weight_updates
            bias_updates = self.momentum * layer.bias_momentums - self.current_learning_rate * layer.dbiases
            layer.bias_momentums = bias_updates
        else:
            weight_updates = -self.current_learning_rate * layer.dweights
            bias_updates = -self.current_learning_rate * layer.dbiases
        layer.weights += weight_updates
        layer.biases += bias_updates
    def post_update_params(self):
        self.iterations += 1

In [24]:
# Create Dense layer with 2 input features and 64 output values
dense1 = Layer_Dense(4096, 10)
# Create ReLU activation (to be used with Dense layer):
activation1 = Activation_ReLU()
# Create second Dense layer with 64 input features (as we take output
# of previous layer here) and 3 output values (output values)
dense2 = Layer_Dense(10, 3)
# Create Softmax classifier's combined loss and activation
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()
# Create optimizer
optimizer = Optimizer_SGD(learning_rate=0.98, decay=1e-3, momentum=0.8)
# Train in loop
for epoch in range(10001):
    # Perform a forward pass of our training data through this layer
    dense1.forward(X)
    # Perform a forward pass through activation function
    # takes the output of first dense layer here
    activation1.forward(dense1.output)
    # Perform a forward pass through second Dense layer
    # takes outputs of activation function of first layer as inputs
    dense2.forward(activation1.output)
    # Perform a forward pass through the activation/loss function
    # takes the output of second dense layer here and returns loss
    loss = loss_activation.forward(dense2.output, y)
    # Calculate accuracy from output of activation2 and targets
    # calculate values along first axis
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions == y)
    if not epoch % 100:
        print(f'epoch: {epoch}, ' +
              f'acc: {accuracy:.3f}, ' +
              f'loss: {loss:.3f}')
    # Backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    # Update weights and biases
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
#     if epoch == 10000:
#         print("Final accuracy: ", accuracy)
#         fig, (ax1,ax2) = plt.subplots(2)
#         x1 = X[y == 0]
#         x2 = X[y == 1]
#         x3 = X[y == 2]

#         ax1.scatter(x1[:, 0], x1[:, 1], c='red', s=40, edgecolor='k')
#         ax1.scatter(x2[:, 0], x2[:, 1], c='blue', s=40, edgecolor='k')
#         ax1.scatter(x3[:, 0], x3[:, 1], c='green', s=40, edgecolor='k')
#         # plt.scatter(X[:, 0], X[:, 1])
#         ax1.set_title("Spiral Data")
#         x1 = X[predictions == 0]
#         x2 = X[predictions == 1]
#         x3 = X[predictions == 2]
#         x4 = X[predictions != y]
#         ax2.scatter(x1[:, 0], x1[:, 1], c='red', s=40, edgecolor='k')
#         ax2.scatter(x2[:, 0], x2[:, 1], c='blue', s=40, edgecolor='k')
#         ax2.scatter(x3[:, 0], x3[:, 1], c='green', s=40, edgecolor='k')
#         ax2.scatter(x4[:, 0], x4[:, 1], c='black', s=40, edgecolor='k')
#         ax2.set_title("predictions")
#         fig.suptitle("Spiral Data and Predictions")
#         # fig.show()
#         plt.show()

epoch: 0, acc: 0.333, loss: 2.335
epoch: 100, acc: 0.333, loss: 1.099
epoch: 200, acc: 0.333, loss: 1.099
epoch: 300, acc: 0.333, loss: 1.099
epoch: 400, acc: 0.333, loss: 1.099
epoch: 500, acc: 0.333, loss: 1.099
epoch: 600, acc: 0.333, loss: 1.099
epoch: 700, acc: 0.333, loss: 1.099
epoch: 800, acc: 0.333, loss: 1.099
epoch: 900, acc: 0.333, loss: 1.099
epoch: 1000, acc: 0.333, loss: 1.099
epoch: 1100, acc: 0.333, loss: 1.099
epoch: 1200, acc: 0.333, loss: 1.099
epoch: 1300, acc: 0.333, loss: 1.099
epoch: 1400, acc: 0.333, loss: 1.099
epoch: 1500, acc: 0.333, loss: 1.099
epoch: 1600, acc: 0.333, loss: 1.099
epoch: 1700, acc: 0.333, loss: 1.099
epoch: 1800, acc: 0.333, loss: 1.099
epoch: 1900, acc: 0.333, loss: 1.099
epoch: 2000, acc: 0.333, loss: 1.099
epoch: 2100, acc: 0.333, loss: 1.099
epoch: 2200, acc: 0.333, loss: 1.099
epoch: 2300, acc: 0.333, loss: 1.099
epoch: 2400, acc: 0.333, loss: 1.099
epoch: 2500, acc: 0.333, loss: 1.099
epoch: 2600, acc: 0.333, loss: 1.099
epoch: 2700, 